In [1]:
import numpy as np
import pandas as pd
import sympy as sp
import networkx as nx
# import matplotlib.pyplot as plt

import random
import string

In [2]:
# behavioral, education, occupation, wealth, family, ...
# IRL might want: crime rate, density, proximity to Rely, proximity to legal firm, ... 
social_capital = pd.read_csv("C:/Users/dvirz/Desktop/Perimeter/Intact/datasets/social_capital_zip.csv")
income_df = pd.read_csv("C:/Users/dvirz/Desktop/Perimeter/Intact/datasets/18zpallagi.csv")
# education_df = pd.read_csv("C:/Users/dvirz/Desktop/Perimeter/Intact/datasets/educational_level_zip")

In [31]:
# # A00100:	Adjust gross income (AGI)
# # income_df.columns
# income_df[['zipcode', 'agi_stub', 'A00100']].head(50)
# # income_df.agi_stub.value_counts()

# # for agi_stub == 1: add column saying "<25k", 2:"25-50k", 3:"50-75k", 4:"75-100k", 5:"100-200k", 6:"200k+"
# income_df['income'] = income_df['agi_stub'].apply(lambda x: "<25k" if x == 1 else "25-50k" if x == 2 else "50-75k" if x == 3 else "75-100k" if x == 4 else "100-200k" if x == 5 else "200k+")
# # new column 'agi_is_correct' for each row, True if A00100 is in the correct range for the agi_stub
# income_df['agi_is_correct'] = income_df.apply(lambda x: (x['agi_stub'] == 1 and 0 <= x['A00100'] < 25000) or \
#                                                         (x['agi_stub'] == 2 and 25000 <= x['A00100'] < 50000) or \
#                                                         (x['agi_stub'] == 3 and 50000 <= x['A00100'] < 75000) or \
#                                                         (x['agi_stub'] == 4 and 75000 <= x['A00100'] < 100000) or \
#                                                         (x['agi_stub'] == 5 and 100000 <= x['A00100'] < 200000) or \
#                                                         (x['agi_stub'] == 6 and 200000 <= x['A00100']), axis=1)

# income_df[['zipcode', 'agi_stub', 'A00100', 'income', 'agi_is_correct']].agi_is_correct.value_counts()
# # almost all false???


# social_capital[social_capital.zip == 1001]
# income_df[income_df.zipcode == 1001]

# print(len(income_df.zipcode), len(income_df.zipcode.unique())) #: print(165935/27558) -> 6 rows per postal code
# income_df[['zipcode', 'agi_stub', 'A00100']]

In [7]:
# income_df[['zipcode', 'agi_stub', 'A00100']].head(50)
# average out income for each zipcode
income_averaged = income_df.groupby('zipcode')['A00100'].mean().to_frame()
income_averaged = income_averaged.reset_index()
income_averaged = income_averaged[1:]
income_averaged = income_averaged.rename(columns={"A00100": "income"})
income_averaged['income'] = income_averaged['income'].apply(lambda x: round(x, -3))
income_averaged['income'] = income_averaged['income'].astype(int)
# as dataframe not series:

In [8]:
# social_capital.pop2018.sum()
# income_df.zipcode
# print(len(social_capital.zip.unique()), len(income_df.zipcode.unique()))
# np.array([item in social_capital.zip.unique() for item in income_df.zipcode.unique()])/

# a = set(social_capital.zip.unique())
# b = set(income_df.zipcode.unique())
# print(len(a.intersection(b)), len(a.difference(b)), len(b.difference(a)))

# merge on postal code, only keep intersection
df = social_capital.merge(income_averaged, left_on='zip', right_on='zipcode', how='inner')

In [11]:
df # now associate with SEM

,zip,county,num_below_p50,pop2018,ec_zip,ec_se_zip,nbhd_ec_zip,ec_grp_mem_zip,ec_high_zip,ec_high_se_zip,...,bias_grp_mem_zip,bias_grp_mem_high_zip,nbhd_bias_zip,nbhd_bias_high_zip,clustering_zip,support_ratio_zip,volunteering_rate_zip,civic_organizations_zip,zipcode,income
0,1001,25013.0,995.787468,17621,0.88157,0.02422,1.51095,1.10210,1.47136,0.01599,...,0.02434,-0.10001,-0.00336,-0.21186,0.105720,0.945260,0.056500,0.010800,1001,91000
1,1002,25015.0,1312.117077,30066,1.18348,0.02227,0.97760,1.23333,1.62290,0.01500,...,0.09856,-0.06421,0.18724,-0.24353,0.103400,0.901630,0.149510,0.036880,1002,147000
2,1005,25027.0,381.519745,4991,1.15543,0.03050,1.46491,1.30756,1.47733,0.01664,...,0.00850,-0.07246,-0.00064,-0.11397,0.105540,0.958370,0.158620,0.021630,1005,25000
3,1007,25015.0,915.396667,14967,1.19240,0.02046,1.17985,1.32294,1.56812,0.01364,...,-0.01188,-0.11464,0.04162,-0.21283,0.103910,0.948730,0.130530,0.016900,1007,103000
4,1008,25013.0,90.721764,1197,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.122236,0.876098,0.051352,0.021194,1008,8000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22669,99824,2110.0,137.286667,2224,1.17545,0.03650,NaN,1.23878,1.49796,0.02885,...,0.04670,-0.10578,NaN,NaN,0.110440,0.905500,0.148520,0.025260,99824,15000
22670,99827,2100.0,323.690765,2565,1.16103,0.02914,0.80469,1.03071,1.42918,0.02810,...,-0.02275,-0.15745,0.01611,-0.21642,0.122400,0.997290,0.100870,0.065760,99827,15000
22671,99833,2195.0,484.405151,3255,1.04721,0.02557,0.92852,1.00921,1.35218,0.02585,...,-0.00256,-0.14167,-0.03406,-0.21637,0.152600,0.999270,0.147380,0.039720,99833,19000
22672,99835,2220.0,790.157898,8738,1.14199,0.01983,1.07357,1.14861,1.42479,0.01759,...,0.00953,-0.12930,0.04966,-0.15935,0.116320,0.995930,0.139090,0.035030,99835,53000


In [5]:
# # problem, bugs_faced should heavily dependent on monthly usage !!

# np.random.seed(42)
# n_samples = 10000

# # trust, outsource, wealth, cultural_group, age, urban_status, education, else

# data = {
#     'wealth': np.random.binomial(1, 0.25, n_samples),
#     }

# df = pd.DataFrame(data)

# df['interactions'] = ...